In [415]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [416]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', required=True, help='cifar10 | lsun | imagenet | folder | lfw | fake')
parser.add_argument('--dataroot', required=True, help='path to dataset')
parser.add_argument('--workers', type=int, help='number of data loading workers', default=2)
parser.add_argument('--batchSize', type=int, default=64, help='input batch size')
parser.add_argument('--imageSize', type=int, default=64, help='the height / width of the input image to network')
parser.add_argument('--nz', type=int, default=100, help='size of the latent z vector')
parser.add_argument('--ngf', type=int, default=64)
parser.add_argument('--ndf', type=int, default=64)
parser.add_argument('--niter', type=int, default=25, help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.0002, help='learning rate, default=0.0002')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for adam. default=0.5')
parser.add_argument('--cuda', action='store_true', help='enables cuda')
parser.add_argument('--ngpu', type=int, default=1, help='number of GPUs to use')
parser.add_argument('--netG', default='', help="path to netG (to continue training)")
parser.add_argument('--netD', default='', help="path to netD (to continue training)")
parser.add_argument('--outf', default='.', help='folder to output images and model checkpoints')
parser.add_argument('--manualSeed', type=int, help='manual seed')

_StoreAction(option_strings=['--manualSeed'], dest='manualSeed', nargs=None, const=None, default=None, type=<type 'int'>, choices=None, help='manual seed', metavar=None)

In [417]:
args = {}

In [418]:
args['batch_size']=100
args['test_batch_size'] =1000

In [419]:
args['manualSeed'] = 5
print("Random Seed: ", args['manualSeed'])

Random Seed:  5


In [420]:
random.seed(args['manualSeed'])
torch.manual_seed(args['manualSeed'])

In [421]:
kwargs = {}
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['test_batch_size'], shuffle=True, **kwargs)

In [422]:
train_use = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args['batch_size'], shuffle=True, **kwargs)

In [423]:
def _calculate_fan_in_and_fan_out(tensor):
    dimensions = tensor.ndimension()
    if dimensions < 2:
        raise ValueError("Fan in and fan out can not be computed for tensor with less than 2 dimensions")

    if dimensions == 2:  # Linear
        fan_in = tensor.size(1)
        fan_out = tensor.size(0)
    else:
        num_input_fmaps = tensor.size(1)
        num_output_fmaps = tensor.size(0)
        receptive_field_size = 1
        if tensor.dim() > 2:
            receptive_field_size = tensor[0][0].numel()
        fan_in = num_input_fmaps * receptive_field_size
        fan_out = num_output_fmaps * receptive_field_size

    return fan_in, fan_out

In [424]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
       # self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
      #  x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [425]:
neural_networks = {}

In [426]:
n_networks = 10
for i in range(n_networks):
    neural_networks[i] = Net()

In [427]:

training_loaders = {}

for i  in range(n_networks):

    training_loaders[i] = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args['batch_size'], shuffle=True, **kwargs)


In [428]:
import math

In [429]:
def weights_init(m, xavier=True, std_given=10., random_seed=1):
    classname = m.__class__.__name__
    random.seed(random_seed)
    torch.manual_seed(random_seed)
    print (classname)
    if classname.find('Conv') != -1:
       # m.weight.data.normal_(0.0, 0.02)
        
        if xavier:
            z = _calculate_fan_in_and_fan_out(m.weight.data)
            std = math.sqrt(2.0 / (z[0] + z[1]))
            m.weight.data.uniform_(-std, std)
        else:
            m.weight.data.uniform_(-std_given, std_given)
    if classname.find('Linear') != -1:
        if xavier:
            z = _calculate_fan_in_and_fan_out(m.weight.data)
            std = math.sqrt(2.0 / (z[0] + z[1]))
            m.weight.data.uniform_(-std, std)
            
            m.bias.data.fill_(0.0)
        else:
            m.weight.data.uniform_(-std_given, std_given)
            m.bias.data.uniform_(-std_given, std_given)



In [430]:
for i in range(1, n_networks - 1):
    magnitudes = 10 ** (- i + 4)
    g = lambda x: weights_init(x, xavier=False, std_given=magnitudes, random_seed=i)
    neural_networks[i].apply(g)

Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net


In [431]:
keep = [0, 1, 2, 3, 4, 5, 9]

In [432]:
z=list(neural_networks[5].parameters())
z[0]

Parameter containing:
(0 ,0 ,.,.) = 
1.00000e-02 *
   6.6050 -7.4778  8.1494  6.3985  8.4021
  -7.6671 -6.7122  4.7584 -9.3347  9.8837
   2.1286  1.2914 -8.5518  3.1868  4.3001
   1.5866  9.6185  3.0032 -8.8673  8.4022
   3.3953 -4.7705 -9.1857  5.7007  9.5032

(1 ,0 ,.,.) = 
1.00000e-02 *
  -8.1931  0.5465  3.5877 -4.7224 -2.1881
  -6.6773 -4.7277 -9.1167 -0.2318  5.9293
   4.8637  9.3941 -8.7825 -1.2304  9.7363
   1.6375 -2.8200 -9.0061  4.6534  7.5917
   8.0187  8.3716  1.9578  7.1274 -2.5171

(2 ,0 ,.,.) = 
1.00000e-02 *
   8.1212 -0.8556 -2.0622 -4.9709  5.3243
   4.0461  6.3608 -6.3907 -5.3170  9.4632
  -5.5504 -5.7032 -2.6756 -3.2321 -6.8007
   8.0813  1.1147  4.5591  7.4174 -6.4800
   7.6339 -5.0095 -1.5921  7.5677  9.2197

(3 ,0 ,.,.) = 
1.00000e-02 *
   8.7785  5.7513  9.2947 -3.6031 -0.6365
  -3.4203 -0.7350 -1.2159  5.3428  6.8734
  -7.2011 -5.5220 -3.0755  1.3368 -3.8818
  -6.2195 -1.8264 -4.8801  0.2753 -7.4026
  -2.4998 -3.6897 -1.3813  7.8573  6.5572

(4 ,0 ,.,.) = 
1.0

In [433]:
args['lr'] = 0.01
args['momentum'] = 0.0

In [434]:
args['cuda'] = False

In [435]:
optimizers = {}

for i in neural_networks:
    optimizers[i] = optim.SGD(neural_networks[i].parameters(), lr=args['lr'], momentum=args['momentum'])

In [436]:
def train(epoch, model, optimizer, train_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
#         if batch_idx % 10 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.data[0]))

    model.eval()
    train_loss = 0
    correct = 0
    for data, target in train_use:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        train_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
    print (train_loss, train_use.dataset)
    train_loss /= len(train_use.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
    return 100. * correct / len(train_loader.dataset)

In [437]:
results = {}
for i in range(n_networks):
    results[i] = []

In [ ]:
for epoch in range(1, 20 + 1):
    for i in keep:
        res = train(epoch, neural_networks[i], optimizers[i], training_loaders[i])
        results[i].append(res)

16484.9489927 <torchvision.datasets.mnist.MNIST object at 0x7f59b57e3c90>

Test set: Average loss: 0.2747, Accuracy: 55161/60000 (92%)

nan <torchvision.datasets.mnist.MNIST object at 0x7f59b57e3c90>

Test set: Average loss: nan, Accuracy: 5923/60000 (10%)

nan <torchvision.datasets.mnist.MNIST object at 0x7f59b57e3c90>

Test set: Average loss: nan, Accuracy: 5923/60000 (10%)

358689.187286 <torchvision.datasets.mnist.MNIST object at 0x7f59b57e3c90>

Test set: Average loss: 5.9782, Accuracy: 5918/60000 (10%)

139539.714401 <torchvision.datasets.mnist.MNIST object at 0x7f59b57e3c90>

Test set: Average loss: 2.3257, Accuracy: 5949/60000 (10%)

18976.6626759 <torchvision.datasets.mnist.MNIST object at 0x7f59b57e3c90>

Test set: Average loss: 0.3163, Accuracy: 54452/60000 (91%)

16496.0890474 <torchvision.datasets.mnist.MNIST object at 0x7f59b57e3c90>

Test set: Average loss: 0.2749, Accuracy: 55023/60000 (92%)

9878.59872007 <torchvision.datasets.mnist.MNIST object at 0x7f59b57e3c90>

Tes